In [ ]:
pip install --upgrade pip

In [ ]:
pip install pandas scikit-learn Keras tensorflow

In [ ]:
pip install tensorflow==2.2.0

# Importing libraries

In [82]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Activation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load dataset and preprocessing of inconclusive values

In [83]:
data = pd.read_csv("dataset/video-game-sales-with-ratings.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [84]:
data.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
data = data.drop('Other_Sales', axis = 1)
data = data.drop('Global_Sales', axis = 1)
data = data.drop('Developer', axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   Critic_Count     8137 non-null   float64
 10  User_Score       10015 non-null  object 
 11  User_Count       7590 non-null   float64
 12  Rating           9950 non-null   object 
dtypes: float64(7), object(6)
memory usage: 1.7+ MB


In [86]:
# Remove all row with nan values.
data = data.dropna(axis = 0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6825 entries, 0 to 16706
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             6825 non-null   object 
 1   Platform         6825 non-null   object 
 2   Year_of_Release  6825 non-null   float64
 3   Genre            6825 non-null   object 
 4   Publisher        6825 non-null   object 
 5   NA_Sales         6825 non-null   float64
 6   EU_Sales         6825 non-null   float64
 7   JP_Sales         6825 non-null   float64
 8   Critic_Score     6825 non-null   float64
 9   Critic_Count     6825 non-null   float64
 10  User_Score       6825 non-null   object 
 11  User_Count       6825 non-null   float64
 12  Rating           6825 non-null   object 
dtypes: float64(7), object(6)
memory usage: 746.5+ KB


In [87]:
# Remover values less than 1.
data = data.loc[data['NA_Sales'] > 1]
data = data.loc[data['EU_Sales'] > 1]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 636
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             258 non-null    object 
 1   Platform         258 non-null    object 
 2   Year_of_Release  258 non-null    float64
 3   Genre            258 non-null    object 
 4   Publisher        258 non-null    object 
 5   NA_Sales         258 non-null    float64
 6   EU_Sales         258 non-null    float64
 7   JP_Sales         258 non-null    float64
 8   Critic_Score     258 non-null    float64
 9   Critic_Count     258 non-null    float64
 10  User_Score       258 non-null    object 
 11  User_Count       258 non-null    float64
 12  Rating           258 non-null    object 
dtypes: float64(7), object(6)
memory usage: 28.2+ KB


In [88]:
data['Name'].value_counts()
game_name = data.Name
data = data.drop('Name', axis = 1)  # Remove column Name
data.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [89]:
# Dataset entry and class (x, y)
data_entry = data.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
sales_na = data.iloc[:, 4].values
sales_eu = data.iloc[:, 5].values
sales_jp = data.iloc[:, 6].values
data_entry[0:5]

array([['Wii', 2006.0, 'Sports', 'Nintendo', 76.0, 51.0, '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', 'Nintendo', 82.0, 73.0, '8.3', 709.0,
        'E'],
       ['Wii', 2009.0, 'Sports', 'Nintendo', 80.0, 73.0, '8', 192.0, 'E'],
       ['DS', 2006.0, 'Platform', 'Nintendo', 89.0, 65.0, '8.5', 431.0,
        'E'],
       ['Wii', 2006.0, 'Misc', 'Nintendo', 58.0, 41.0, '6.6', 129.0, 'E']],
      dtype=object)

In [90]:
one_hot_encoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
data_entry = one_hot_encoder.fit_transform(data_entry).toarray()

# Model configuration for training

In [91]:
input_layer = Input(shape=(61,))
hidden_layer_1 = Dense(units=32, activation='sigmoid')(input_layer)
hidden_layer_2 = Dense(units=32, activation='sigmoid')(hidden_layer_1)
exit_layer_1 = Dense(units=1, activation='linear')(hidden_layer_2)
exit_layer_2 = Dense(units=1, activation='linear')(hidden_layer_2)
exit_layer_3 = Dense(units=1, activation='linear')(hidden_layer_2)

regressor = Model(inputs=input_layer, outputs=[exit_layer_1, exit_layer_2, exit_layer_3])
regressor.compile(optimizer='adam', loss='mse')

# Trains the model for a fixed number of epochs

In [ ]:
regressor.fit(data_entry, [sales_na, sales_eu, sales_jp], epochs=5000, batch_size=100)

In [80]:
result_na, result_eu, result_jp = regressor.predict(data_entry)

In [99]:
sales_na[0:15]

array([41.36, 15.68, 15.61, 11.28, 13.96, 14.44,  9.71,  8.92, 15.  ,
        9.01,  7.02,  4.74,  9.66,  8.41,  3.43])

In [101]:
result_na[0:5]

array([[30.405054 ],
       [15.155758 ],
       [13.335852 ],
       [ 6.1656075],
       [ 4.739014 ]], dtype=float32)

# Save neural network and weights

In [103]:
# Save neural network
regressor_json = regressor.to_json()

with open('network/regression_games.json', 'w') as json_file:
    json_file.write(regressor_json)

In [105]:
# Save weights
regressor.save_weights('model/regression_games.h5')